### Setup

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import subprocess
import astropy.table
import astropy.io.fits as fits
import numpy as np
from copy import deepcopy
import random
from astropy.table import Table
import os 
import matplotlib.pyplot as plt 
import fitsio 

In [3]:
#directories that would be using 
locations = dict(
WLD = '/Users/Ismael/code/lensing/WeakLensingDeblending/',
SMFF = '/Users/Ismael/code/lensing/SMFF/',
aegis = '/Users/Ismael/aegis/data/',
SEx = '/Users/Ismael/aegis/data/sextractor_runs/',
aegis_slac = '/nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/data',
temp_data = '/Users/Ismael/temp_data'
)
os.chdir(locations['WLD']) #set this to interested directory 

In [4]:
def mad(arr):
    """ Median Absolute Deviation: a "Robust" version of standard deviation.
        Indices variabililty of the sample.
        https://en.wikipedia.org/wiki/Median_absolute_deviation 
    """
    arr = np.ma.array(arr).compressed() # should be faster to not use masked arrays.
    med = np.median(arr)
    return np.median(np.abs(arr - med))

### prepare input 

In [5]:
#files 
#g1,g2=0,0
#g1,g2=.01,.01
#g1,g2=-.01,-.01
#g1,g2=.02,.02
#g1,g2=-.02,-.02
files = dict(
final_fits1 = 'Users/Ismael/aegis/data/fits_files/final_fits1.fits', 
final_fits2 = 'Users/Ismael/aegis/data/fits_files/final_fits2.fits', 
final_fits3 = 'Users/Ismael/aegis/data/fits_files/final_fits3.fits', 
final_fits4 = 'Users/Ismael/aegis/data/fits_files/final_fits4.fits', 
final_fits5 = 'Users/Ismael/aegis/data/fits_files/final_fits5.fits', 
)

files_temps = dict() 
for f in files: 
    l = files[f].split("/")
    temp_file = '{0}{1}'.format('/Users/Ismael/temp_data/',l[-1])
    files_temps[f] = temp_file

    
files_slac = dict()
for f in files: 
    l = files[f].split("/")
    index_data = l.index("aegis")
    str_slac = "/".join(l[index_data:])
    slac_file = '{0}{1}'.format('/nfs/slac/g/ki/ki19/deuce/AEGIS/ismael/',str_slac)
    files_slac[f] = slac_file

### Samples 

In [6]:
#some interesting subsets of the simulation
iso_gal = lambda cat: cat[cat['purity'] > .98] #isolated galaxies
grp_gal = lambda cat: cat[cat['purity'] <= .98] #galaxies in a group of 2 or more. 

#'good' galaxies satisfy the reasonable criteria below.
good = lambda cat: cat[(cat['snr_grpf'] > 6) & (cat['sigma_m'] > .2)]

#gold sample galaxies 
gold = lambda cat: cat[(cat['ab_mag'] < 25.3)] 

#ambiguity of blends. 
ambig = lambda cat: cat[cat['ambig_blend'] == True ]
not_ambig = lambda cat: cat[cat['ambig_blend'] == False ]
detected = lambda cat: cat[cat['match'] != -1]
not_detected = lambda cat: cat[cat['match'] == -1]

#cuts 
cut_biasiso = lambda cat,bias_cut: cat[(np.absolute(cat['bias_g1']) < bias_cut) & (np.absolute(cat['bias_g2']) < bias_cut)]
cut_biasgrp = lambda cat,bias_cut: cat[(np.absolute(cat['bias_g1_grp']) < bias_cut) & (np.absolute(cat['bias_g2_grp']) < bias_cut)]
down_cut = lambda cat,param,cut: cat[cat[param] < cut]
up_cut = lambda cat,param,cut: cat[cat[param] > cut]
abs_cut = lambda cat,param,cut,point=0: cat[np.absolute(cat[param] - point) < cut]
unphysical_iso = lambda cat: cat[(np.absolute(cat['bias_g1']) > 1.) | (abs(cat['bias_g2']) > 1.)]
unphysical_grp = lambda cat: cat[(np.absolute(cat['bias_g1_grp']) > 1.) | (np.absolute(cat['bias_g2_grp']) > 1.)]

#more specific 
detc_and_notambig = lambda cat: cat[(cat['ambig_blend'] == False) & (cat['match'] != -1)]
notdetc_and_notambig = lambda cat: cat[(cat['ambig_blend'] == False) & (cat['match'] == -1)]
detc_and_ambig = lambda cat: cat[(cat['ambig_blend'] == True) & (cat['match'] != -1)]
notdetc_and_ambig = lambda cat: cat[(cat['ambig_blend'] == True) & (cat['match'] == -1)]
best = detc_and_notambig

### What is gold/good fractions for galaxies in section001.fits?

In [7]:
f = fits.open('/Users/Ismael/temp_data/final_fits.fits')
table = Table.read(f,hdu=1)

In [13]:
print 'statistics for golden objects:'
print 'not ambiguously blended detected objects: {0:.3f}%'.format(float(len(gold(detc_and_notambig(table))))/len(gold(table)))
print 'ambiguously blended detected objects: {0:.3f}%'.format(float(len(gold(detc_and_ambig(table))))/len(gold(table)))
print 'not ambiguously blended and undetected objects: {0:.3f}%'.format(float(len(gold(notdetc_and_notambig(table))))/len(gold(table)))
print 'ambiguously blended and undetected objects: {0:.3f}%'.format(float(len(gold(notdetc_and_ambig(table))))/len(gold(table)))
print 
print 'statistics for good objects:'
print 'not ambiguously blended detected objects: {0:.3f}%'.format(float(len(good(detc_and_notambig(table))))/len(good(table)))
print 'ambiguously blended detected objects: {0:.3f}%'.format(float(len(good(detc_and_ambig(table))))/len(good(table)))
print 'not ambiguously blended and undetected objects: {0:.3f}%'.format(float(len(good(notdetc_and_notambig(table))))/len(good(table)))
print 'ambiguously blended and undetected objects: {0:.3f}%'.format(float(len(good(notdetc_and_ambig(table))))/len(good(table)))





statistics for golden objects:
not ambiguously blended detected objects: 0.658%
ambiguously blended detected objects: 0.222%
not ambiguously blended and undetected objects: 0.056%
ambiguously blended and undetected objects: 0.063%

statistics for good objects:
not ambiguously blended detected objects: 0.768%
ambiguously blended detected objects: 0.150%
not ambiguously blended and undetected objects: 0.061%
ambiguously blended and undetected objects: 0.021%
